In [2]:
# ==========================================================
# High-Fidelity Artwork Analyzer v13 (Visual+Functional)
# ==========================================================
# 100% LOCAL (OpenCV, NumPy, Pillow). No internet, no extras.
#
# What you get (per image):
#   /<image>_analysis/
#     ├─ overlay_final_composite.png  (single image with clear overlays + legend)
#     ├─ curatorial_summary.txt       (micro + narrative, informed + encouraging)
#     └─ metrics.json                 (all numeric metrics for inspection)
#
# Overlays (color-coded):
#   • Focal point (🔴 red rings)
#   • Subject region (🟣 magenta contour + soft glow)
#   • Saliency field (soft red haze)
#   • Texture energy (🟠 warm amber haze from Gabor)
#   • Local color-contrast ΔE proxy (🟣 magenta haze)
#   • Composition lines (cyan/green/yellow by angle)
#   • Rule-of-thirds & Golden ratio nodes (white / yellow)
#   • Light direction arrow (teal) & Vanishing point (yellow dot)
#   • Hue-bar + Orientation-bar + textual legend strip
# ==========================================================

import os, json, time, textwrap
import numpy as np
import cv2
from PIL import Image
from math import atan2, degrees

# -----------------------------
# Utils
# -----------------------------
def load_rgb(path):
    img = Image.open(path).convert("RGB")
    return np.array(img)

def ensure_dir(p): os.makedirs(p, exist_ok=True)

def to_uint8(img_f32):
    return np.clip(img_f32*255.0, 0, 255).astype(np.uint8)

def safe_gamma(img_f32):
    # Keep image visually balanced across light/dark paintings
    m = float(img_f32.mean())
    if m < 0.20:  # too dark → brighten
        img_f32 = np.power(img_f32, 0.6)
    elif m > 0.85:  # too bright → compress
        img_f32 = np.power(img_f32, 1.2)
    return np.clip(img_f32, 0, 1)

# -----------------------------
# ΔE2000 (perceptual contrast)
# -----------------------------
def ciede2000(Lab1, Lab2):
    L1, a1, b1 = Lab1
    L2, a2, b2 = Lab2
    avg_L = (L1 + L2) / 2.0
    C1 = np.sqrt(a1*a1 + b1*b1)
    C2 = np.sqrt(a2*a2 + b2*b2)
    avg_C = (C1 + C2) / 2.0
    G = 0.5 * (1 - np.sqrt((avg_C**7) / (avg_C**7 + 25**7)))
    a1p = (1+G)*a1; a2p = (1+G)*a2
    C1p = np.sqrt(a1p*a1p + b1*b1); C2p = np.sqrt(a2p*a2p + b2*b2)
    avg_Cp = (C1p + C2p) / 2.0
    h1p = np.degrees(np.arctan2(b1, a1p)) % 360
    h2p = np.degrees(np.arctan2(b2, a2p)) % 360
    dLp = L2 - L1; dCp = C2p - C1p
    dhp = h2p - h1p
    if C1p*C2p == 0:
        dhp = 0
    else:
        if dhp > 180: dhp -= 360
        elif dhp < -180: dhp += 360
    dHp = 2*np.sqrt(C1p*C2p)*np.sin(np.radians(dhp)/2.0)
    avg_Lp = (L1 + L2)/2.0
    if C1p*C2p == 0:
        avg_hp = h1p + h2p
    else:
        if abs(h1p - h2p) > 180:
            avg_hp = (h1p + h2p + 360)/2.0 if (h1p + h2p) < 360 else (h1p + h2p - 360)/2.0
        else:
            avg_hp = (h1p + h2p)/2.0
    T = (1
         - 0.17*np.cos(np.radians(avg_hp - 30))
         + 0.24*np.cos(np.radians(2*avg_hp))
         + 0.32*np.cos(np.radians(3*avg_hp + 6))
         - 0.20*np.cos(np.radians(4*avg_hp - 63)))
    d_ro = 30*np.exp(-((avg_hp - 275)/25)**2)
    Rc = 2*np.sqrt((avg_Cp**7)/(avg_Cp**7 + 25**7))
    Sl = 1 + 0.015*(avg_Lp - 50)**2 / np.sqrt(20 + (avg_Lp - 50)**2)
    Sc = 1 + 0.045*avg_Cp
    Sh = 1 + 0.015*avg_Cp*T
    Rt = -np.sin(np.radians(2*d_ro))*Rc
    dE = np.sqrt((dLp/Sl)**2 + (dCp/Sc)**2 + (dHp/Sh)**2 + Rt*(dCp/Sc)*(dHp/Sh))
    return float(dE)

# -----------------------------
# Color (deep)
# -----------------------------
def circular_histogram_hue(h_degrees, bins=36):
    hist, edges = np.histogram(h_degrees, bins=bins, range=(0,360))
    hist = hist.astype(np.float32); hist /= (hist.sum() + 1e-6)
    centers = (edges[:-1] + edges[1:]) / 2.0
    return hist, centers

def analyze_color_deep(img, k=6):
    H, W = img.shape[:2]
    lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
    Z = lab.reshape(-1,3).astype(np.float32)
    crit = (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 60, 0.3)
    _r, labels, centers = cv2.kmeans(Z, k, None, crit, 10, cv2.KMEANS_PP_CENTERS)
    labels2 = labels.reshape(H, W)
    centers = centers.astype(np.float32)

    rgb_centers = cv2.cvtColor(centers.reshape(-1,1,3).astype(np.uint8),
                               cv2.COLOR_Lab2RGB).reshape(-1,3)
    palette = [tuple(map(int, c)) for c in rgb_centers]

    counts = np.bincount(labels.flatten(), minlength=k).astype(np.float32)
    masks = [(labels2 == i).astype(np.uint8)*255 for i in range(k)]
    cluster_area_frac = counts / (H*W + 1e-6)
    spatial_cohesion = float(cluster_area_frac.max())
    p = cluster_area_frac[cluster_area_frac > 0]
    palette_entropy = -float(np.sum(p * np.log2(p)))

    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    Hh, Sh, Vh = hsv[:,:,0].astype(np.float32), hsv[:,:,1].astype(np.float32), hsv[:,:,2].astype(np.float32)
    hue_deg = (Hh * 2.0) % 360.0
    hue_hist, hue_centers = circular_histogram_hue(hue_deg, bins=36)
    sat_mean, sat_std = float(Sh.mean()), float(Sh.std())
    val_mean, val_std = float(Vh.mean()), float(Vh.std())

    a, b = centers[:,1]-128, centers[:,2]-128
    hues_c = (np.degrees(np.arctan2(b, a)) + 360) % 360
    warm_ratio = float(((hues_c < 90) | (hues_c > 330)).sum()/len(hues_c))
    cool_ratio = float(((hues_c > 180) & (hues_c < 300)).sum()/len(hues_c))
    harmony = "mixed"
    if np.ptp(hues_c) < 35: harmony = "analogous"
    elif any(abs(hues_c[i]-hues_c[j])%360 > 150 for i in range(len(hues_c)) for j in range(i+1,len(hues_c))):
        harmony = "complementary"

    dEs = []
    for i in range(len(centers)):
        for j in range(i+1, len(centers)):
            dEs.append(ciede2000(centers[i], centers[j]))
    mean_dE = float(np.mean(dEs)) if dEs else 0.0
    max_dE = float(np.max(dEs)) if dEs else 0.0

    # ΔE proxy field: gradient magnitude in Lab (multi-channel)
    labf = lab.astype(np.float32)
    gxL = cv2.Sobel(labf[:,:,0], cv2.CV_32F, 1, 0); gyL = cv2.Sobel(labf[:,:,0], cv2.CV_32F, 0, 1)
    gxa = cv2.Sobel(labf[:,:,1], cv2.CV_32F, 1, 0); gya = cv2.Sobel(labf[:,:,1], cv2.CV_32F, 0, 1)
    gxb = cv2.Sobel(labf[:,:,2], cv2.CV_32F, 1, 0); gyb = cv2.Sobel(labf[:,:,2], cv2.CV_32F, 0, 1)
    grad_mag = np.sqrt((gxL**2 + gyL**2) + (gxa**2 + gya**2) + (gxb**2 + gyb**2))
    contrast_map = cv2.GaussianBlur(grad_mag, (0,0), 1.2)
    contrast_map = (contrast_map - contrast_map.min())/(contrast_map.max()-contrast_map.min()+1e-6)

    return dict(
        palette_rgb=palette, masks=masks,
        color_variability=float(np.std(centers,0).mean()),
        warm_ratio=warm_ratio, cool_ratio=cool_ratio, harmony=harmony,
        hue_hist=hue_hist, hue_centers=hue_centers,
        sat_mean=sat_mean, sat_std=sat_std, val_mean=val_mean, val_std=val_std,
        palette_entropy=palette_entropy, spatial_cohesion=spatial_cohesion,
        mean_deltaE=mean_dE, max_deltaE=max_dE, contrast_map=contrast_map
    )

# -----------------------------
# Texture (LBP + Gabor)
# -----------------------------
def lbp_basic(gray):
    H,W = gray.shape
    out = np.zeros((H-2, W-2), dtype=np.uint8)
    c = gray[1:-1,1:-1]
    code = np.zeros_like(c, dtype=np.uint8)
    shifts = [(-1,-1,7),(-1,0,6),(-1,1,5),(0,1,4),(1,1,3),(1,0,2),(1,-1,1),(0,-1,0)]
    for dy,dx,bit in shifts:
        nb = gray[1+dy:H-1+dy, 1+dx:W-1+dx]
        code |= ((nb >= c).astype(np.uint8) << bit)
    return code

def gabor_energy(gray, orientations=6, scales=(3,5,7)):
    H,W = gray.shape
    energy_maps = []
    angles = np.linspace(0, 180, orientations, endpoint=False)
    combined = np.zeros((H,W), np.float32)
    ori_energy = []
    for theta in angles:
        theta_rad = np.deg2rad(theta)
        e_sum = np.zeros((H,W), np.float32)
        for k in scales:
            kernel = cv2.getGaborKernel((21,21), sigma=k, theta=theta_rad, lambd=10, gamma=0.5, psi=0)
            resp = cv2.filter2D(gray, cv2.CV_32F, kernel)
            e_sum += resp*resp
        ori_energy.append(e_sum.mean())
        combined = np.maximum(combined, e_sum)
        energy_maps.append(e_sum)
    ori_energy = np.array(ori_energy, dtype=np.float32)
    dominant_idx = int(np.argmax(ori_energy))
    dominant_angle = float(angles[dominant_idx])
    anisotropy = float((ori_energy.max() - ori_energy.min()) / (ori_energy.mean()+1e-6))
    comb_norm = (combined - combined.min())/(combined.max()-combined.min()+1e-6)
    return angles, ori_energy, comb_norm, anisotropy, dominant_angle

def analyze_texture_nuanced(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY).astype(np.uint8)
    lbp = lbp_basic(gray)
    hist, _ = np.histogram(lbp, bins=256, range=(0,256))
    hist = hist.astype(np.float32); hist /= (hist.sum()+1e-6)
    lbp_entropy = -float(np.sum(hist * np.log2(hist + 1e-12)))
    lbp_roughness = float(lbp.std())
    angles, ori_energy, energy_map, anisotropy, dom_angle = gabor_energy(gray, orientations=6, scales=(3,5,7))
    return dict(
        lbp_entropy=lbp_entropy,
        lbp_roughness=lbp_roughness,
        gabor_angles=angles, gabor_energy=ori_energy,
        gabor_map=energy_map, gabor_anisotropy=anisotropy, gabor_dom_angle=dom_angle
    )

# -----------------------------
# Saliency (frequency-based)
# -----------------------------
def spectral_saliency(grayf):
    fft=np.fft.fft2(grayf)
    loga=np.log(np.abs(fft)+1e-8)
    spec=np.exp((loga-cv2.blur(loga,(3,3)))+1j*np.angle(fft))
    sal=np.abs(np.fft.ifft2(spec))**2
    sal=cv2.GaussianBlur(sal,(9,9),2)
    sal=(sal-sal.min())/(sal.max()-sal.min()+1e-6)
    return (sal*255).astype(np.uint8)

def subject_from_saliency(sal):
    H,W=sal.shape
    _,th=cv2.threshold(sal,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    cnts,_=cv2.findContours(th,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    if cnts:
        x,y,w,h=cv2.boundingRect(max(cnts,key=cv2.contourArea))
    else: x,y,w,h=W//4,H//4,W//2,H//2
    mask=np.zeros((H,W),np.uint8); cv2.rectangle(mask,(x,y),(x+w,y+h),255,-1)
    return dict(mask=mask,bbox=(x,y,w,h))

def analyze_saliency(img):
    g=cv2.cvtColor(img,cv2.COLOR_RGB2GRAY).astype(np.float32)
    sal=spectral_saliency(g)
    sub=subject_from_saliency(sal)
    Y,X=np.indices(sal.shape); w=sal.astype(np.float32)+1
    cx,cy=int((X*w).sum()/w.sum()),int((Y*w).sum()/w.sum())
    return dict(saliency_map=sal,focal=(cx,cy),
                subject_mask=sub["mask"],subject_bbox=sub["bbox"])

# -----------------------------
# Composition (lines, thirds, golden, symmetry, balance)
# -----------------------------
def analyze_composition_nuanced(img, focal_xy):
    g=cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    H,W=g.shape
    edges=cv2.Canny(g,100,200)
    linesP=cv2.HoughLinesP(edges,1,np.pi/180,100,minLineLength=min(H,W)//10,maxLineGap=15)
    lines=[tuple(map(int,l[0])) for l in linesP] if linesP is not None else []
    fx,fy=focal_xy
    thirds_pts=[(W/3,H/3),(2*W/3,H/3),(W/3,2*H/3),(2*W/3,2*H/3)]
    golden=0.618
    golden_pts=[(golden*W,golden*H),((1-golden)*W,golden*H),
                (golden*W,(1-golden)*H),((1-golden)*W,(1-golden)*H)]
    diag=np.hypot(W,H)
    thirds_delta=min(np.hypot(fx-x,fy-y) for (x,y) in thirds_pts)/(diag+1e-6)
    golden_delta=min(np.hypot(fx-x,fy-y) for (x,y) in golden_pts)/(diag+1e-6)
    g_f = g.astype(np.float32)/255.0
    left = g_f[:, :W//2]
    right = g_f[:, W - left.shape[1]:]
    right_flipped = np.fliplr(right)
    mse = float(np.mean((left - right_flipped)**2))
    symmetry = float(1.0 - min(1.0, mse*4.0))
    lw,rw=float(np.sum(edges[:,:W//2])),float(np.sum(edges[:,W//2:]))
    balance=(rw-lw)/(lw+rw+1e-6)
    return dict(
        lines=lines, edge_balance=balance, thirds_delta=thirds_delta,
        golden_delta=golden_delta, symmetry=symmetry, vanishing_point=(W//2,H//2)
    )

# -----------------------------
# Illumination
# -----------------------------
def illumination(img):
    L=cv2.cvtColor(img,cv2.COLOR_RGB2LAB)[:,:,0].astype(np.float32)
    mu,sig=L.mean(),L.std()+1e-6
    skew=float((((L-mu)/sig)**3).mean())
    gx,gy=cv2.Sobel(L,cv2.CV_32F,1,0),cv2.Sobel(L,cv2.CV_32F,0,1)
    hi=(L>np.percentile(L,90)).astype(np.uint8)
    if hi.sum()>0:
        gx,gy=(gx*hi).sum(),(gy*hi).sum()
        ang=float((degrees(atan2(-gy,-gx))+360)%360)
    else: ang=None
    return dict(lightness_skew=skew,light_direction_deg=ang)

# -----------------------------
# Legend helpers
# -----------------------------
def make_hue_bar(hue_hist, width=420, height=50):
    bar = np.zeros((height, width, 3), np.uint8)
    for x in range(width):
        h = int((x/width)*179)
        col = np.uint8([[[h, 200, 220]]])
        rgb = cv2.cvtColor(col, cv2.COLOR_HSV2RGB)[0,0]
        bar[:, x] = rgb
    bins = len(hue_hist); bin_w = max(1, width // bins)
    hist = hue_hist/(hue_hist.max()+1e-6)
    for i, v in enumerate(hist):
        x1, x2 = i*bin_w, min(width-1, (i+1)*bin_w-1)
        h_px = int(v*(height-10))
        cv2.rectangle(bar, (x1, height-1), (x2, height-1-h_px), (0,0,0), -1)
    return bar

def make_orientation_bar(angles, energies, width=420, height=50):
    bar = np.full((height, width, 3), 240, np.uint8)
    if energies is None or len(energies)==0: return bar
    e = np.array(energies, dtype=np.float32)
    e = (e - e.min())/(e.max()-e.min()+1e-6)
    for a, v in zip(angles, e):
        x = int((a/180.0)*width)
        h_px = int(v*(height-10))
        cv2.rectangle(bar, (x-2, height-1), (x+2, height-1-h_px), (60,60,60), -1)
    cv2.putText(bar, "0°", (5,18), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1)
    cv2.putText(bar, "90°", (width//2-15,18), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1)
    cv2.putText(bar, "180°", (width-50,18), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1)
    return bar

# -----------------------------
# Curatorial Text Generator
# -----------------------------
def generate_curatorial_summary(M):
    C, T, P, I = (M["color"], M["texture"], M["composition"], M["illumination"])
    if C["harmony"] == "analogous":
        color_tone = "a restrained palette that breathes with quiet unity"
    elif C["harmony"] == "complementary":
        color_tone = "a lively chromatic counterpoint that animates the surface"
    else:
        color_tone = "a flexible palette balancing contrast and cohesion"
    if C["mean_deltaE"] > 25:
        contrast_phrase = "marked by pronounced contrast"
    elif C["mean_deltaE"] > 10:
        contrast_phrase = "held in a moderate register of contrast"
    else:
        contrast_phrase = "gentle in chromatic transitions"
    texture_line = ("Texture discloses a directional force (anisotropy "
                    f"{T['gabor_anisotropy']:.2f}) with a prevailing angle near "
                    f"{T['gabor_dom_angle']:.0f}°.") if T["gabor_anisotropy"]>0.5 else \
                   "Texture remains even and diffused, emphasizing tonal balance."
    comp_tone = ("The focal point leans toward classical proportion (golden Δ="
                 f"{P['golden_delta']:.3f}).") if P["golden_delta"]<0.12 else \
                ("The focal point aligns intuitively with rule-of-thirds (Δ="
                 f"{P['thirds_delta']:.3f}).") if P["thirds_delta"]<0.12 else \
                "The composition resists convention, inviting open visual movement."
    sym_phrase = ("Axial calm arises from notable symmetry."
                  if P["symmetry"]>0.8 else
                  "Partial symmetry tempers dynamism."
                  if P["symmetry"]>0.5 else
                  "Asymmetry lends motion and immediacy.")
    balance_phrase = ("Edge-weight remains stable."
                      if abs(P["edge_balance"])<0.1 else
                      "A deliberate imbalance animates the frame.")
    if I["light_direction_deg"] is not None:
        illum_phrase = f"Light enters around {int(I['light_direction_deg'])}°, guiding attention."
    else:
        illum_phrase = "Light diffuses evenly, reinforcing tonal unity."
    if I["lightness_skew"] > 0.5:
        mood = "luminous and open"
    elif I["lightness_skew"] < -0.5:
        mood = "dark-hued and contemplative"
    else:
        mood = "balanced and calm"

    micro = textwrap.fill(
        " ".join([
            f"Color: {C['harmony']} harmony, mean ΔE={C['mean_deltaE']:.1f}, max ΔE={C['max_deltaE']:.1f}, "
            f"entropy={C['palette_entropy']:.2f}, spatial={C['spatial_cohesion']:.2f}.",
            f"Texture: LBP-H={T['lbp_entropy']:.2f}, rough={T['lbp_roughness']:.1f}, "
            f"anisotropy={T['gabor_anisotropy']:.2f}, dom={T['gabor_dom_angle']:.0f}°.",
            f"Composition: thirds Δ={P['thirds_delta']:.3f}, golden Δ={P['golden_delta']:.3f}, "
            f"sym={P['symmetry']:.2f}, balance={P['edge_balance']:.2f}.",
            f"Light: skew={I['lightness_skew']:.2f}, direction={I['light_direction_deg']}."
        ]),
        width=90
    )

    curatorial = textwrap.fill(
        " ".join([
            f"The work operates within {color_tone}, {contrast_phrase}.",
            texture_line, comp_tone, sym_phrase, balance_phrase, illum_phrase,
            f"The overall mood feels {mood}.",
            "Together these elements invite a slow, attentive gaze, where color, gesture, and structure"
            " are given room to breathe."
        ]),
        width=90
    )
    return micro, curatorial

# -----------------------------
# Visualization / Composite with Legend
# -----------------------------
def make_composite(img, M):
    base = img.astype(np.float32)/255.0
    H, W = base.shape[:2]

    # Layers with soft alpha
    # Color clusters (tinted)
    color_layer = np.zeros_like(base)
    for m, c in zip(M["color"]["masks"], M["color"]["palette_rgb"]):
        if m.sum()==0: continue
        mask = cv2.GaussianBlur((m>0).astype(np.float32), (0,0), 6)
        color_layer += np.dstack([mask]*3) * (np.array(c)/255.0)
    color_layer *= 0.18

    # Texture (Gabor energy → warm amber)
    gmap = M["texture"]["gabor_map"]
    gmap = cv2.GaussianBlur(gmap, (0,0), 1.2)
    gmap = (gmap - gmap.min())/(gmap.max()-gmap.min()+1e-6)
    tex_layer = np.dstack([gmap, gmap*0.45, 0*gmap]) * 0.28

    # Local color-contrast ΔE proxy (magenta)
    contrast = M["color"]["contrast_map"]
    contrast = cv2.GaussianBlur(contrast, (0,0), 1.2)
    contrast = (contrast - contrast.min())/(contrast.max()-contrast.min()+1e-6)
    de_layer = np.dstack([contrast*0.35, 0*contrast, contrast*0.35]) * 0.30

    # Saliency (soft red)
    sal = M["saliency"]["saliency_map"].astype(np.float32)
    sal = cv2.resize(sal, (W, H))
    sal = cv2.normalize(sal, None, 0, 1, cv2.NORM_MINMAX)
    sal_layer = np.dstack([sal*0.55, sal*0.2, sal*0.2]) * 0.24

    # Lines (cyan/green/yellow by angle)
    line_layer = np.zeros_like(base)
    for (x1,y1,x2,y2) in M["composition"]["lines"][:180]:
        ang = (np.rad2deg(np.arctan2(y2-y1, x2-x1)) + 180) % 180
        # vertical-ish (blue), horizontal-ish (green), diagonal (yellow)
        col = (0,1,0) if 60<ang<120 else (0,0,1) if ang<30 or ang>150 else (1,1,0)
        cv2.line(line_layer,(x1,y1),(x2,y2),(col[2],col[1],col[0]),1,cv2.LINE_AA)
    line_layer = cv2.GaussianBlur(line_layer,(0,0),2)*0.45

    # Blend
    comp = base*0.85 + color_layer + tex_layer + de_layer + sal_layer + line_layer
    comp = np.clip(comp, 0, 1)

    # Subject contour + soft glow + focal rings
    comp_bgr = (comp[:,:,::-1]*255).astype(np.uint8).copy()
    mask = M["saliency"]["subject_mask"]
    cnts,_ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if cnts:
        # soft glow
        glow = np.zeros_like(comp_bgr)
        cv2.drawContours(glow, cnts, -1, (255, 0, 255), thickness=cv2.FILLED)
        glow = cv2.GaussianBlur(glow, (0,0), 15)
        comp_bgr = cv2.addWeighted(glow, 0.10, comp_bgr, 0.90, 0)
        # edge
        cv2.drawContours(comp_bgr, cnts, -1, (255, 0, 255), 2)

    fx, fy = M["saliency"]["focal"]
    for r in range(6, 28, 6):
        cv2.circle(comp_bgr, (int(fx), int(fy)), r, (0,0,255), 1)

    # Thirds & Golden nodes
    thirds = [(W//3, H//3), (2*W//3, H//3), (W//3, 2*H//3), (2*W//3, 2*H//3)]
    for (x,y) in thirds:
        cv2.circle(comp_bgr, (int(x),int(y)), 3, (240,240,240), -1)
    golden = 0.618
    golds = [(golden*W,golden*H),((1-golden)*W,golden*H),
             (golden*W,(1-golden)*H),((1-golden)*W,(1-golden)*H)]
    for (x,y) in golds:
        cv2.circle(comp_bgr, (int(x),int(y)), 3, (0,255,255), -1)

    # Vanishing point + Light arrow
    vp = M["composition"].get("vanishing_point")
    if vp: cv2.circle(comp_bgr, tuple(map(int, vp)), 5, (0,255,255), -1)
    ang = M["illumination"].get("light_direction_deg")
    if ang is not None:
        cx, cy = W-60, 60; r = min(H,W)//3
        ex, ey = int(cx - r*np.cos(np.radians(ang))), int(cy + r*np.sin(np.radians(ang)))
        cv2.arrowedLine(comp_bgr, (cx,cy), (ex,ey), (0,150,255), 3, tipLength=0.1)

    comp_rgb = comp_bgr[:,:,::-1].astype(np.float32)/255.0

    # Legend (dark bar) with labels and insets
    legend_h = 160
    legend = np.zeros((legend_h, W, 3), np.uint8)
    f = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(legend,"Legend",(20,36),f,1.0,(255,255,255),2)

    # swatches
    sw=[((140,10),(175,40),(0,0,255),"Focal"),
        ((190,10),(225,40),(255,0,255),"Subject"),
        ((240,10),(275,40),(0,255,255),"Lines"),
        ((290,10),(325,40),(255,128,0),"Texture"),
        ((340,10),(375,40),(255,0,255),"ΔE"),
        ((390,10),(425,40),(255,255,255),"Thirds"),
        ((440,10),(475,40),(0,255,255),"Golden"),
        ((490,10),(525,40),(0,150,255),"Light→")]
    for (x1,y1),(x2,y2),col,label in sw:
        cv2.rectangle(legend,(x1,y1),(x2,y2),col,-1)
        cv2.putText(legend,label,(x2+8,y2+22),f,0.55,(255,255,255),1)

    # key metrics text blocks
    C = M["color"]; T = M["texture"]; P = M["composition"]; I = M["illumination"]
    txt1 = (f"color: harm={C['harmony']}  meanΔE={C['mean_deltaE']:.1f}  "
            f"maxΔE={C['max_deltaE']:.1f}  entropy={C['palette_entropy']:.2f}  "
            f"spatial={C['spatial_cohesion']:.2f}")
    txt2 = (f"texture: LBP-H={T['lbp_entropy']:.2f}  rough={T['lbp_roughness']:.1f}  "
            f"anisotropy={T['gabor_anisotropy']:.2f}  dom={T['gabor_dom_angle']:.0f}°")
    txt3 = (f"composition: thirdsΔ={P['thirds_delta']:.3f}  goldenΔ={P['golden_delta']:.3f}  "
            f"sym={P['symmetry']:.2f}  balance={P['edge_balance']:.2f}")
    cv2.putText(legend, txt1, (20, 75), f, 0.60, (220,220,220), 1)
    cv2.putText(legend, txt2, (20,105), f, 0.60, (220,220,220), 1)
    cv2.putText(legend, txt3, (20,135), f, 0.60, (220,220,220), 1)

    # insets on right: hue bar + orientation bar
    hue_bar = make_hue_bar(C["hue_hist"], width=min(500, max(360, W//2)), height=50)
    ori_bar = make_orientation_bar(T["gabor_angles"], T["gabor_energy"],
                                   width=min(500, max(360, W//2)), height=50)
    hb_h, hb_w = hue_bar.shape[:2]
    ob_h, ob_w = ori_bar.shape[:2]
    x0 = max(0, W - max(hb_w, ob_w) - 20)

    # clip hue/ori bars if they exceed legend width
    hue_bar_resized = hue_bar[:, :min(hb_w, W - x0)]
    ori_bar_resized = ori_bar[:, :min(ob_w, W - x0)]

    legend[10:10 + hb_h, x0:x0 + hue_bar_resized.shape[1]] = hue_bar_resized
    legend[10 + hb_h + 5:10 + hb_h + 5 + ob_h, x0:x0 + ori_bar_resized.shape[1]] = ori_bar_resized


    final = np.vstack([to_uint8(safe_gamma(comp_rgb)), legend])
    final = final.astype(np.uint8)
    return final

# -----------------------------
# Pipeline
# -----------------------------
def analyze_artwork_high_fidelity(path):
    start = time.time()
    img = load_rgb(path)

    color = analyze_color_deep(img, k=6)
    texture = analyze_texture_nuanced(img)
    sal = analyze_saliency(img)
    comp = analyze_composition_nuanced(img, focal_xy=sal["focal"])
    illum = illumination(img)

    M = dict(color=color, texture=texture, saliency=sal, composition=comp, illumination=illum)

    # Build composite
    composite = make_composite(img, M)

    # Outputs
    out_dir = os.path.splitext(path)[0] + "_analysis"
    ensure_dir(out_dir)
    out_png = os.path.join(out_dir, "overlay_final_composite.png")
    cv2.imwrite(out_png, composite)

    # Curatorial text
    micro, narrative = generate_curatorial_summary(M)
    out_txt = os.path.join(out_dir, "curatorial_summary.txt")
    with open(out_txt, "w", encoding="utf-8") as f:
        f.write("--- MICRO SUMMARY ---\n")
        f.write(micro + "\n\n--- CURATORIAL SUMMARY ---\n")
        f.write(narrative + "\n")

    # Metrics JSON (safe serialization for NumPy types)
    def make_json_safe(obj):
        if isinstance(obj, dict):
            return {k: make_json_safe(v) for k, v in obj.items()}
        elif isinstance(obj, (list, tuple)):
            return [make_json_safe(x) for x in obj]
        elif isinstance(obj, (np.generic, np.number)):
            return obj.item()
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return obj

    metrics_py = make_json_safe(M)
    out_json = os.path.join(out_dir, "metrics.json")
    with open(out_json, "w", encoding="utf-8") as f:
        json.dump(metrics_py, f, indent=2)

# -----------------------------
# Run
# -----------------------------
if __name__=="__main__":
    # ⬇️ Set your image path here:
    IMAGE_PATH = "/Users/alievanayasso/Documents/SlowMA/JMB.jpeg"
    analyze_artwork_high_fidelity(IMAGE_PATH)
